In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import datetime
import os
from feature_engineering.tensor_features import develop_features, floating_conv
from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader
from path_finder import path_sorter

2024-10-30 13:30:28.993184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 13:30:29.000769: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-30 13:30:29.083563: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-30 13:30:29.156181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 13:30:29.228051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [17]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../Project materials(1)")

Retrieving training data...


In [18]:
print(parsed_data.keys())

print(parsed_data.head())

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'portId', 'etaParsed', 'UN_LOCODE', 'ISO',
       'portLongitude', 'portLatitude'],
      dtype='object')
                 time    cog   sog  rot  heading  navstat  latitude  \
0 2024-01-01 00:00:25  284.0   0.7    0       88        0 -34.74370   
1 2024-01-01 00:00:36  109.6   0.0   -6      347        1   8.89440   
2 2024-01-01 00:01:45  111.0  11.0    0      112        0  39.19065   
3 2024-01-01 00:03:11   96.4   0.0    0      142        1 -34.41189   
4 2024-01-01 00:03:51  214.0  19.7    0      215        0  35.88379   

   longitude                  vesselId                    portId  \
0  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4   -5.91636  61e9f41bb93

In [20]:
testing_set = parsed_data[parsed_data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
print(testing_set.shape)
first_time = testing_set["time"].iloc[0]
last_time = testing_set["time"].iloc[-1]
print(first_time)
print(last_time)
print(testing_set.head(-10))

(1550, 15)
2024-01-01 00:00:25
2024-05-01 12:41:57
                       time    cog   sog  rot  heading  navstat  latitude  \
0       2024-01-01 00:00:25  284.0   0.7    0       88        0 -34.74370   
2654    2024-01-01 06:09:08   92.8  14.2    0       90        0 -35.16787   
2927    2024-01-01 06:38:19   90.5  14.3    0       88        0 -35.16863   
3072    2024-01-01 06:58:55   88.2  14.3    0       86        0 -35.16805   
3119    2024-01-01 07:15:56   88.3  12.3    0       86        0 -35.16715   
...                     ...    ...   ...  ...      ...      ...       ...   
1419749 2024-04-29 15:51:11  237.7  12.9    0      233        0  41.41531   
1419925 2024-04-29 16:02:10  234.7  13.6    0      229        0  41.39277   
1420119 2024-04-29 16:33:01  239.3  14.1    0      235        0  41.32875   
1420487 2024-04-29 17:14:23  237.6  15.0    0      236        0  41.23535   
1420660 2024-04-29 17:32:47  237.8  15.4    0      236        0  41.19335   

         longitude      

In [21]:
index_data = parsed_data.copy()
index_data.set_index("time", inplace=True)
print(index_data.head())


                 time    cog   sog  rot  heading  navstat  latitude  \
0 2024-01-01 00:00:25  284.0   0.7    0       88        0 -34.74370   
1 2024-01-01 00:00:36  109.6   0.0   -6      347        1   8.89440   
2 2024-01-01 00:01:45  111.0  11.0    0      112        0  39.19065   
3 2024-01-01 00:03:11   96.4   0.0    0      142        1 -34.41189   
4 2024-01-01 00:03:51  214.0  19.7    0      215        0  35.88379   

   longitude                  vesselId                    portId  \
0  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

            etaParsed UN_LOCODE ISO  portLongitude  portLatitude  
0 2024-01-09 23:00:00     CLSAI  CL     -71.618889      -33.5875  
1 2024-12-29 20:00:00     PAPT

In [43]:

import loadBar

def fill_with_proximity(df):
    # Forward fill first
    df_ffill = df.ffill()
    
    # Backward fill next
    df_bfill = df.bfill()
    # Create a mask for original missing values
    mask = df.isna()

    # Create a new DataFrame to hold the results
    filled_df = df.copy()

    # Iterate over each missing value
    for i in range(len(df)):
        if mask.iloc[i, 0]:  # Check if the value is missing
            # Get the current index (timestamp)
            current_time = df.index[i]

            # Get the last known value index (timestamp)
            last_known_index = df_ffill.first_valid_index()
            if last_known_index is not None:
                distance_to_ffill = (current_time - last_known_index).total_seconds()  # distance to last known

            # Get the next known value index (timestamp)
            next_known_index = df_bfill.first_valid_index()
            if next_known_index is not None:
                distance_to_bfill = (next_known_index - current_time).total_seconds()  # distance to next known

            # Fill based on proximity
            if (last_known_index is not None and distance_to_ffill < distance_to_bfill) or next_known_index is None:
                filled_df.iloc[i] = df_ffill.iloc[i]
            else:
                filled_df.iloc[i] = df_bfill.iloc[i]

    return filled_df



def resampler(df, sorting_column, freq):
    unique_ids = df[sorting_column].unique()
    final_df = pd.DataFrame()
    partial_list = []

    for i in range(len(unique_ids)):
        loadBar.load_bar(len(unique_ids),i+1)
        resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()
        resample_partial = fill_with_proximity(resample_partial)
        partial_list.append(resample_partial)

    for chunk in partial_list:
        final_df = pd.concat([final_df,chunk])
    
    return final_df

resampled_data_h = resampler(index_data, "vesselId", "h")
resampled_data_20min = resampler(index_data, "vesselId", "20T")

resampled_data_h.to_csv('resampled_data_h.csv')
resampled_data_20min.to_csv('resampled_data_20min.csv')


[===================-] 100.0% complete


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


...

/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


[===================-] 100.0% complete


/tmp/ipykernel_25648/2420621250.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()


In [11]:
#We have two resampled and therefore regular datasets, now we need to make our time series into a supervised problem.
#First I need to change navstat into a categorical feature:


# Define categories based on ranges or discrete values
pretty_20m = pd.read_csv("resampled_data_20min.csv")

pretty_h = pd.read_csv("resampled_data_h.csv")

navstat_unique = pretty_h["navstat"].unique()

pretty_20m["navstat"] = pd.Categorical(pretty_20m["navstat"], categories=navstat_unique, ordered=True)

pretty_h["navstat"] = pd.Categorical(pretty_h["navstat"], categories=navstat_unique, ordered=True)


# Let's make dummys 

pretty_h = pd.get_dummies(pretty_h, columns=["navstat"], drop_first=True)
pretty_20m = pd.get_dummies(pretty_20m, columns = ["navstat"], drop_first=True)

pretty_20m.set_index("time", inplace=True)
pretty_h.set_index("time", inplace=True)


In [23]:
print(pretty_h.head())
print(pretty_h.shape)

                       cog   sog  rot  heading  latitude  longitude  \
time                                                                  
2024-01-01 00:00:00  284.0   0.7  0.0     88.0 -34.74370   -57.8513   
2024-01-01 01:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 02:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 03:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 04:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   

                                     vesselId                    portId  \
time                                                                      
2024-01-01 00:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 01:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 02:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 03:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 04:00:00  61e9f3a8b937134a3c4bfdf7  61d37

In [41]:
#Make time series into supervised problem
def make_supervised(df, forecast_columns, sorting_column, input_window=1, output_window=1):
    """
    Converts a multivariate time series dataframe into a supervised learning problem.
    
    Parameters:
    df (pd.DataFrame): The original dataframe with time series data.
    forecast_columns (list): A list of column names to forecast.
    input_window (int): The number of past observations to use as features.
    output_window (int): The number of steps to forecast into the future.
    
    Returns:
    pd.DataFrame: A new dataframe with supervised learning format.
    """
    # Create a list to hold the transformed value
    df_holder = []
    
    #Put in a for loop here where you iterate over all IDs, to make sure things get correct
    unique_sorts = df[sorting_column].unique()
    
    #Iterate through all IDs
    for sorts in unique_sorts:
        df_supervised = pd.DataFrame()
        sort_df = df[df[sorting_column] == sorts]

        #Iterate through all columns for input features
        for col in sort_df.columns: 
            for i in range(input_window, 0, -1):
                df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
            df_supervised[f"{col}_t"] = sort_df[col]
            

    # Create columns for forecast (target) with forward shift
        for col in forecast_columns:
            for j in range(output_window, 0,-1):
                df_supervised[f"{col}_t+{j}"] = sort_df[col].shift(-j)
        
        df_holder.append(df_supervised)
    

    
    df_new = pd.DataFrame()
    
    for chunk in df_holder:
        df_new = pd.concat([df_new, chunk])
    # Remove rows with NaN values caused by the shifting process
    df_new.dropna(inplace=True)
    
    return df_new

supervised_h = make_supervised(pretty_h, ["latitude", "longitude"],"vesselId" , 3, 2)

supervised_h.columns

/tmp/ipykernel_7830/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
/tmp/ipykernel_7830/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
/tmp/ipykernel_7830/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To 

Index(['cog_t-3', 'cog_t-2', 'cog_t-1', 'cog_t', 'sog_t-3', 'sog_t-2',
       'sog_t-1', 'sog_t', 'rot_t-3', 'rot_t-2',
       ...
       'navstat_13.0_t-1', 'navstat_13.0_t', 'navstat_9.0_t-3',
       'navstat_9.0_t-2', 'navstat_9.0_t-1', 'navstat_9.0_t', 'latitude_t+2',
       'latitude_t+1', 'longitude_t+2', 'longitude_t+1'],
      dtype='object', length=112)

In [42]:
print(supervised_h.head(-10))

                     cog_t-3  cog_t-2  cog_t-1  cog_t  sog_t-3  sog_t-2  \
time                                                                      
2024-01-01 03:00:00    284.0     88.2     88.2   88.2      0.7     14.3   
2024-01-01 04:00:00     88.2     88.2     88.2   88.2     14.3     14.3   
2024-01-01 05:00:00     88.2     88.2     88.2   88.2     14.3     14.3   
2024-01-01 06:00:00     88.2     88.2     88.2   88.2     14.3     14.3   
2024-01-01 07:00:00     88.2     88.2     88.2   86.8     14.3     14.3   
...                      ...      ...      ...    ...      ...      ...   
2024-05-03 13:00:00    194.8    124.2     80.8   58.3     13.3      1.9   
2024-05-03 14:00:00    124.2     80.8     58.3   61.9      1.9     18.1   
2024-05-03 15:00:00     80.8     58.3     61.9   59.1     18.1     17.7   
2024-05-03 16:00:00     58.3     61.9     59.1   61.6     17.7     19.2   
2024-05-03 17:00:00     61.9     59.1     61.6   59.0     19.2     18.9   

                     sog